In [1]:
import pandas as pd
import numpy as np

In [2]:
def find_s_algorithm(training_data):
    hypothesis = None
    for i, row in training_data.iterrows():
        if row["Enjoy Sport"] == "Yes":
            if hypothesis is None:
                hypothesis = row[:-1]
            else:
                for col in training_data.columns[:-1]:
                    if hypothesis[col] != row[col]:
                        hypothesis[col] = "?"
    return hypothesis

def predict_g_algorithm(g, instance):
    for hypothesis in g:
        for i, val in enumerate(instance):
            if hypothesis[i] != "?" and hypothesis[i] != val:
                break
        else:
            return "Yes"
    return "No"


In [3]:
def find_g_algorithm(training_data):
    num_features = len(training_data.columns) - 1
    g = [["?" for _ in range(num_features)] for _ in range(num_features)]
    positive_instances = training_data[training_data["Enjoy Sport"] == "Yes"]
    if len(positive_instances) == 0:
        return []

    specific_h = positive_instances.iloc[0][:-1].tolist()

    for idx, row in training_data.iterrows():
        instance = row[:-1].tolist()
        if row["Enjoy Sport"] == "Yes":
            for i in range(num_features):
                if instance[i] != specific_h[i]:
                    specific_h[i] = "?"
                    g[i][i] = "?"

        else:
            for i in range(num_features):
                if instance[i] != specific_h[i]:
                    g[i][i] = specific_h[i]
                else:
                    g[i][i] = "?"

    g = [h for h in g if h != ["?" for _ in range(num_features)]]

    return g

def predict_g_algorithm(g, instance):
    for hypothesis in g:
        for i, val in enumerate(instance):
            if hypothesis[i] != "?" and hypothesis[i] != val:
                break
        else:
            return "Yes"
    return "No"


In [4]:
def candidate_elimination(training_data):
    s_boundary = find_s_algorithm(training_data)
    g_boundary = find_g_algorithm(training_data)
    return s_boundary, g_boundary

In [5]:
def is_more_general_than_s(hypothesis, s_boundary):
    for h, s in zip(hypothesis, s_boundary):
        if h != '?' and h != s and s!= '?':
            return False
        return True

def is_more_specific_than_g(hypothesis, g_boundary):
    return any(all(g[i] == "?" or g[i] == h for i, h in enumerate(hypothesis)) for g in g_boundary)


In [6]:
def find_version_space(s_boundary, g_boundary, all_possible_values):
    version_space = []
    s_list = s_boundary.to_list() if isinstance(s_boundary, pd.Series) else s_boundary

    def generate_hypothesis(current, depth=0):
        if depth == len(s_list):
            if is_more_general_than_s(current, s_list) and is_more_specific_than_g(current, g_boundary):
                version_space.append(current[:])
            return
        for val in all_possible_values[depth] + ['?']:
            current[depth] = val
            generate_hypothesis(current, depth + 1)

    initial_hypothesis = ['?' for _ in range(len(s_list))]
    generate_hypothesis(initial_hypothesis)
    return version_space


In [7]:
def generate_valid_hypotheses(s_boundary, g_boundary):

    valid_hypotheses = []

    # Convert s_boundary to list if it's a pandas Series
    if hasattr(s_boundary, 'tolist'):
        s_boundary = s_boundary.tolist()

    def is_more_general_than_s(h):
        """Check if hypothesis h is more general than S boundary"""
        return all((s_val == '?' or h_val == '?' or s_val == h_val)
                  for h_val, s_val in zip(h, s_boundary))

    def is_more_specific_than_g(h):
        """Check if hypothesis h is more specific than any G boundary"""
        return any(all((g_val == '?' or h_val == g_val)
                      for h_val, g_val in zip(h, g))
                  for g in g_boundary)

    def backtrack(current, pos):
        if pos == len(s_boundary):
            if is_more_general_than_s(current) and is_more_specific_than_g(current):
                valid_hypotheses.append(current[:])
            return

        # If S has a specific value, we can either keep it or generalize to '?'
        if s_boundary[pos] != '?':
            # Try keeping the specific value
            current[pos] = s_boundary[pos]
            backtrack(current, pos + 1)

            # Try generalizing to '?' if allowed by G
            if any(g[pos] == '?' for g in g_boundary):
                current[pos] = '?'
                backtrack(current, pos + 1)
        else:
            # If S has '?', we must keep it as '?'
            current[pos] = '?'
            backtrack(current, pos + 1)

    # Start with empty hypothesis
    initial = ['?' for _ in range(len(s_boundary))]
    backtrack(initial, 0)

    return valid_hypotheses

In [8]:
def predict_with_confidence(instance, valid_hypotheses):
    if not valid_hypotheses:
        return "No", 0.0

    matches = sum(1 for h in valid_hypotheses if
                 all(h[i] == "?" or h[i] == instance[i] for i in range(len(instance))))

    confidence = matches / len(valid_hypotheses)
    return "Yes" if confidence > 0.5 else "No", confidence

In [9]:
training_data = {
        'Sky': ['Sunny', 'Sunny', 'Rainy', 'Sunny'],
        'Temperature': ['Warm', 'Warm', 'Cold', 'Warm'],
        'Humidity': ['Normal', 'High', 'High', 'High'],
        'Wind': ['Strong', 'Strong', 'Strong', 'Strong'],
        'Water': ['Warm', 'Warm', 'Warm', 'Cool'],
        'Forecast': ['Same', 'Same', 'Change', 'Change'],
        'Enjoy Sport': ['Yes', 'Yes', 'No', 'Yes']
    }

training_data = pd.DataFrame(training_data)

s_boundary, g_boundary = candidate_elimination(training_data)

In [19]:
pd.DataFrame(s_boundary).T

,Sky,Temperature,Humidity,Wind,Water,Forecast
0,Sunny,Warm,?,Strong,?,?


In [11]:
pd.DataFrame(g_boundary)

,0,1,2,3,4,5
0,Sunny,?,?,?,?,?
1,?,Warm,?,?,?,?


In [12]:
s_boundary, g_boundary

(Sky             Sunny
 Temperature      Warm
 Humidity            ?
 Wind           Strong
 Water               ?
 Forecast            ?
 Name: 0, dtype: object,
 [['Sunny', '?', '?', '?', '?', '?'], ['?', 'Warm', '?', '?', '?', '?']])

In [13]:
all_possible_values = [['Sunny', 'Rainy'], ['Warm', 'Cold'], ['Normal', 'High'], ['Strong'], ['Warm', 'Cool'], ['Same', 'Change']]

In [14]:
version_space = find_version_space(s_boundary, g_boundary, all_possible_values)
pd.DataFrame(version_space)

,0,1,2,3,4,5
0,Sunny,Warm,Normal,Strong,Warm,Same
1,Sunny,Warm,Normal,Strong,Warm,Change
2,Sunny,Warm,Normal,Strong,Warm,?
3,Sunny,Warm,Normal,Strong,Cool,Same
4,Sunny,Warm,Normal,Strong,Cool,Change
...,...,...,...,...,...,...
211,?,Warm,?,?,Cool,Change
212,?,Warm,?,?,Cool,?
213,?,Warm,?,?,?,Same
214,?,Warm,?,?,?,Change


In [15]:
valid_hypotheses = generate_valid_hypotheses(s_boundary, g_boundary)
valid_hypotheses

[['Sunny', 'Warm', '?', 'Strong', '?', '?'],
 ['Sunny', 'Warm', '?', '?', '?', '?'],
 ['Sunny', '?', '?', 'Strong', '?', '?'],
 ['Sunny', '?', '?', '?', '?', '?'],
 ['?', 'Warm', '?', 'Strong', '?', '?'],
 ['?', 'Warm', '?', '?', '?', '?']]

In [16]:
test_instances = [
        ['Sunny', 'Warm', 'Normal', 'Strong', 'Cool', 'Change'],
        ['Sunny', 'Cold', 'Normal', 'Strong', 'Warm', 'Same']
    ]

In [17]:
results = []
for instance in test_instances:
    prediction, confidence = predict_with_confidence(instance, valid_hypotheses)
    results.append({
        'Instance': instance,
        'Prediction': prediction,
        'Confidence': confidence
    })

In [18]:
results

[{'Instance': ['Sunny', 'Warm', 'Normal', 'Strong', 'Cool', 'Change'],
  'Prediction': 'Yes',
  'Confidence': 1.0},
 {'Instance': ['Sunny', 'Cold', 'Normal', 'Strong', 'Warm', 'Same'],
  'Prediction': 'No',
  'Confidence': 0.3333333333333333}]